# Simulations with TAMU data and [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

This is a basic simulation example using the [TAMU Cases](https://electricgrids.engr.tamu.edu/).

## Dependencies

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
using Dates

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from TAMU data
We have included some of the TAMU cases (with truncated time series info)
in the PowerSystemCaseBuilder testing, so we can just use that.

In [4]:
sys = build_system(PSYTestSystems, "tamu_ACTIVSg2000_sys")

horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);

┌ Info: Building new system tamu_ACTIVSg2000_sys from raw data
└   sys_descriptor.raw_data = "/home/runner/.julia/packages/PowerSystemCaseBuilder/XPh1v/data"
[ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
[ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
[ Info: Switched shunt converted to fixed shunt, with default value gs=0.0
[ Info: angmin and angmax values are 0, widening these values on branch 2243 to +/- 60.0 deg.
[ Info: angmin and angmax values are 0, widening these values on branch 1881 to +/- 60.0 deg.
[ Info: angmin and angmax values are 0, widening these values on branch 1907 to +/- 60.0 deg.
[ Info: angmin and angmax values are 0, widening these values on branch 2923 to +/- 60.0 deg.
[ Info: angmin and angmax values are 0, widening these values on branch 599 to +/- 60.0 deg.
[ Info: angmin and angmax values are 0, widening these values on

## Run a PCM
note that the TAMU data doesn't contain startup and shutdown costs, or minimum up/down
time limits, so a UC problem merely respects minmum generation levels.

In [5]:
sim_folder = mkpath(joinpath(pkgpath, "TAMU-sim"))
problems = SimulationProblems(
    UC = OperationsProblem(
        template_unit_commitment(transmission = CopperPlatePowerModel),
        sys,
        optimizer = solver,
    ),
)
intervals = Dict("UC" => (Hour(24), Consecutive()))
DA_sequence = SimulationSequence(
    problems = problems,
    intervals = intervals,
    ini_cond_chronology = IntraProblemChronology(),
)

┌ Warning: No Feedforward Chronologies have been defined. This configuration assummes that there is no information passing between problems
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/E5zjV/src/core/simulation_sequence.jl:103


Feed Forward Chronology
-----------------------

Initial Condition Chronology
----------------------------

1



### Define and build a simulation

In [6]:
sim = Simulation(
    name = "TAMU-test",
    steps = 3,
    problems = problems,
    sequence = DA_sequence,
    simulation_folder = sim_folder,
)

build!(sim)

┌ Info: 
│  ────────────────────────────────────────────────────────────────────────────────
│                                         Time                    Allocations      
│                                ───────────────────────   ────────────────────────
│        Tot / % measured:            9.34s / 100.0%           1.79GiB / 100.0%    
│ 
│  Section               ncalls     time    %tot     avg     alloc    %tot      avg
│  ────────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    9.34s  100.0%   9.34s   1.79GiB  100.0%  1.79GiB
│    Build Problems           1    7.80s   83.5%   7.80s   1.64GiB   91.4%  1.64GiB
│      Problem UC             1    7.52s   80.5%   7.52s   1.62GiB   90.7%  1.62GiB
│        PowerSystems...      1    1.39s   14.8%   1.39s    292MiB   15.9%   292MiB
│        PowerSystems...      1    1.33s   14.2%   1.33s    111MiB    6.1%   111MiB
│        PowerSimulat...      1    220ms    2.4%   220ms   40.4M

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

### Execute the simulation

In [7]:
execute!(sim)

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

### Load and analyze results

In [8]:
sim_results = SimulationResults(sim);
uc_results = get_problem_results(sim_results, "UC")
read_realized_variables(uc_results, names = [:On__ThermalStandard])[:On__ThermalStandard]

[ Info: checking integrity of simulation_store.h5
[ Info: reading variables from data store


,DateTime,generator-1004-1,generator-1006-1,generator-1009-1,generator-1011-1
,DateTime,Float64,Float64,Float64,Float64
1,2016-01-01T00:00:00,1.0,1.0,1.0,1.0
2,2016-01-01T01:00:00,1.0,1.0,1.0,1.0
3,2016-01-01T02:00:00,1.0,1.0,1.0,1.0
4,2016-01-01T03:00:00,1.0,1.0,1.0,1.0
5,2016-01-01T04:00:00,1.0,1.0,1.0,1.0
6,2016-01-01T05:00:00,1.0,1.0,1.0,1.0
7,2016-01-01T06:00:00,1.0,1.0,1.0,1.0
8,2016-01-01T07:00:00,1.0,1.0,1.0,1.0
9,2016-01-01T08:00:00,1.0,1.0,1.0,1.0


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*